In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('data/train.csv')

In [4]:
data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [5]:
from sklearn import preprocessing

In [6]:
test = pd.read_csv('data/test.csv', index_col=0)

In [7]:
test.head()

,Dates,DayOfWeek,PdDistrict,Address,X,Y
Id,,,,,,
0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [8]:
def append_districts(distr_column):
    enc = preprocessing.LabelBinarizer()
    enc.fit(distr_column)
    districts = enc.transform(distr_column)
    return districts

In [9]:
data.Dates = pd.to_datetime(data.Dates)
test.Dates = pd.to_datetime(test.Dates)

In [11]:
def conv_street(street):
    if street.find('/') != -1:
        return map(str.strip, street.split('/'))
    
    pos = street.find('Block of ')
    if pos != -1:
        
        return [street[pos+9:]]
    
    return [street]

In [12]:
streets = set()
for x in data.Address[0:10]:
    streets |= set(conv_street(x))

In [13]:
def append_streets(adr_column, streets):
    streets_cols = np.zeros((len(adr_column),len(streets)), dtype=int)
    for i, street in enumerate(streets):
        for j, address in enumerate(adr_column):
            if address.find(street) != -1:
                streets_cols[j,i] = 1
    return streets_cols

In [14]:
append_streets(data.Address,streets)[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]])

In [15]:
def append_day_of_week(day_column):
    days = {'Sunday':0, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6}
    days_cols = np.zeros((len(day_column),len(days)),dtype=int)
    for i, day in enumerate(day_column):
        days_cols[i,days[day]] = 1
    return days_cols

In [17]:
def append_times(date_column):
    hours_cols = np.zeros((len(date_column),24),dtype=int)
    day_cols = np.zeros((len(date_column),31),dtype=int)
    month_cols = np.zeros((len(date_column),12),dtype=int)
    year_cols = np.zeros((len(date_column),13),dtype=int)
    for i, date in enumerate(date_column):
        hours_cols[i,date.hour] = 1
        day_cols[i,date.day-1] = 1
        month_cols[i,date.month-1] = 1
        year_cols[i,date.year-2003] = 1
    return np.hstack((hours_cols,day_cols,month_cols,year_cols))

In [105]:
from sklearn import decomposition

In [114]:
pca = decomposition.PCA(n_components=2)

def prepare_coords(coords,fit=False):
    if(fit):
        pca.fit(coords)
    return pca.transform(coords)

In [14]:
category_enc = preprocessing.LabelEncoder()
category_enc.fit(data.Category)

LabelEncoder()

In [115]:
X_train = np.hstack((prepare_coords(data[['X', 'Y']].values, True),
                     append_times(data.Dates),
                     append_day_of_week(data.DayOfWeek),
                     append_streets(data.Address,streets),
                     append_districts(data.PdDistrict)))

In [116]:
y_train = category_enc.transform(data.Category)

In [117]:
X_test = np.hstack((prepare_coords(test[['X', 'Y']].values),
                    append_times(test.Dates),
                    append_day_of_week(test.DayOfWeek),
                    append_streets(test.Address,streets),
                    append_districts(test.PdDistrict)))

In [22]:
import xgboost as xgb

In [47]:
param = {
    'nthread'             : 4,
    'booster'             : 'gbtree',
    'objective'           : 'multi:softprob',
    'eval_metric'         : 'mlogloss',
    'num_class'           : len(list(category_enc.classes_)),
    'eta'                 : 1.0,
    #'gamma'               : 0,
    'max_depth'           : 6,
    #'min_child_weigth'    : 1,
    'max_delta_step'      : 1
    #'subsample'           : 1,
    #'colsample_bytree'    : 1,
    #'early.stop.round'    : 5

}

In [ ]:
#gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)

In [120]:
gbm3 = xgb.XGBClassifier(n_estimators=200,
                         nthread=4,
                         objective='mulit:softprob',
                         max_depth=6,
                         silent=False,
                         max_delta_step=1)
gbm3

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=1, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=200, nthread=4,
       objective='mulit:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [122]:
gbm3.fit(X_train, y_train, eval_metric='mlogloss', n)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=1, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=200, nthread=4,
       objective='mulit:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [123]:
y_pred = gbm3.predict(X_test)

In [124]:
y_pred_proba = gbm3.predict_proba(X_test)

In [88]:
y_pred.shape

(884262,)

In [17]:
y_pred_proba.shape

(39, 1768524)

In [127]:
np.save("models/gbm3_y_pred.npy", y_pred)
np.save("models/gbm3_y_pred_proba.npy", y_pred_proba)

In [10]:
y_pred_proba = np.load("models/gbm3_y_pred_proba.npy")

In [18]:
predictions = np.split(y_pred_proba, 2, axis=1)[1].T

In [128]:
from sklearn.externals import joblib
joblib.dump(gbm3, 'models/gbm_3.pkl') 

['models/gbm_3.pkl', 'models/gbm_3.pkl_01.npy']

In [12]:
def build_predicts_matrix(pred, labels):
    m = np.zeros((len(pred),len(labels)))
    for i,x in enumerate(pred):
        m[i,x] = 1
    return m

In [20]:
def save_predicts(name, predictions, index, columns, float_format='%.6f'):
    df = pd.DataFrame(predictions, index=index, columns=columns)
    df.index.name = 'Id'
    df.to_csv('./output/{}.csv.gz'.format(name), compression='gzip', float_format=float_format)

In [21]:
save_predicts('xgb3', predictions, test.index, category_enc.classes_)